In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import metal
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel
from dataset import QQPDataset, RTEDataset, WNLIDataset, MNLIDataset

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
# all_tasks = ["RTE", "WNLI", "QQP", "MNLI"]

# task_classes = {"RTE":2, "WNLI":2, "QQP":2, "MNLI":3}

# task_name = all_tasks[2]

In [4]:
# if task_name == "RTE":
#     train_ds = RTEDataset(split='train', bert_model='bert-base-uncased', max_len=128)
#     train_dl, dev_dl = train_ds.get_dataloader(split_prop=0.8, batch_size=32)

#     test_ds = RTEDataset(split='dev', bert_model='bert-base-uncased', max_len=128)
#     test_dl = test_ds.get_dataloader(batch_size=32)

# if task_name == "WNLI":
#     train_ds = WNLIDataset(split='train', bert_model='bert-base-uncased', max_len=128)
#     train_dl, dev_dl = train_ds.get_dataloader(split_prop=0.8, batch_size=32)

#     test_ds = WNLIDataset(split='dev', bert_model='bert-base-uncased', max_len=128)
#     test_dl = test_ds.get_dataloader(batch_size=32)

# if task_name == "QQP":
#     train_ds = QQPDataset(split='train', bert_model='bert-base-uncased', max_len=128)
#     train_dl, dev_dl = train_ds.get_dataloader(split_prop=0.8, batch_size=32)

#     test_ds = QQPDataset(split='dev', bert_model='bert-base-uncased', max_len=128)
#     test_dl = test_ds.get_dataloader(batch_size=32)

# if task_name == "MNLI":
#     train_ds = MNLIDataset(split='train', bert_model='bert-base-uncased', max_len=128)
#     train_dl, dev_dl = train_ds.get_dataloader(split_prop=0.8, batch_size=32)

#     test_ds = MNLIDataset(split='dev_matched', bert_model='bert-base-uncased', max_len=128)
#     test_dl = test_ds.get_dataloader(batch_size=32)
    
# dataloaders = {
#     'train': train_dl,
#     'valid': dev_dl,
#     'test': test_dl
# }

In [5]:
# import torch.nn as nn

# model = 'bert-base-uncased'

# class BertEncoder(nn.Module):
#     def __init__(self, dropout=0.1):
#         super(BertEncoder, self).__init__()
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased')
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, data):
#         tokens, segments, mask = data
#         _, hidden_layer = self.bert_model(tokens, segments, mask, output_all_encoded_layers=False)
#         hidden_layer = self.dropout(hidden_layer)

#         return hidden_layer

In [6]:
# class LinearLayer(nn.Module):
#     def __init__(self, input_size, output_size):
#         super(LinearLayer, self).__init__()
#         self.linear = nn.Linear(input_size, output_size)

#     def forward(self, x):
#         return self.linear(x)

In [7]:
# hidden_bert_size=768

# def get_task_head(name):
#     return LinearLayer(hidden_bert_size, task_classes[name])


In [8]:
from metal.mmtl.task import Task
from metal.mmtl.glue.glue_tasks import create_tasks

# dataloaders = get_dataloaders(task_name)
# task_head = get_task_head(task_name)


# task = Task(task_name, dataloaders, BertEncoder(), task_head)
# task = create_task('RTE')
# task = create_task('QQP')
# task = create_task('WNLI')
# task = create_task('MNLI')
# task = create_task('MRPC')

# task = create_task('RTE_SAN')

task_names = [
    "RTE_SAN",
]


tasks = create_tasks(task_names,
                     max_datapoints=1000,
                     dl_kwargs={
                         "batch_size": 16
                     }
                    )

Loading RTE Dataset


In [9]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=3, 
    lr=5e-5,
    progress_bar=False,
    log_every=0.25,
    score_every=0.25,
    checkpoint_best=True,
    #checkpoint_metric=task.name + "/valid/accuracy",
    #checkpoint_metric_mode="max",
    verbose=True,
#     device="cuda",
)

Beginning train loop.
Expecting a total of _approximately_ 800 examples and 50 batches per epoch from 1 tasks.
[0.26 epo]: TRAIN:[loss=0.693] VALID:[RTE/accuracy=0.500]
Saving model at iteration 0.26 with best (min) score 0.693
[0.52 epo]: TRAIN:[loss=0.704] VALID:[RTE/accuracy=0.525]
[0.78 epo]: TRAIN:[loss=0.685] VALID:[RTE/accuracy=0.565]
Saving model at iteration 0.78 with best (min) score 0.685
[1.04 epo]: TRAIN:[loss=0.668] VALID:[RTE/accuracy=0.555]
Saving model at iteration 1.04 with best (min) score 0.668
[1.30 epo]: TRAIN:[loss=0.584] VALID:[RTE/accuracy=0.625]
Saving model at iteration 1.3 with best (min) score 0.584
[1.56 epo]: TRAIN:[loss=0.561] VALID:[RTE/accuracy=0.600]
Saving model at iteration 1.56 with best (min) score 0.561
[1.82 epo]: TRAIN:[loss=0.591] VALID:[RTE/accuracy=0.620]
[2.08 epo]: TRAIN:[loss=0.546] VALID:[RTE/accuracy=0.645]
Saving model at iteration 2.08 with best (min) score 0.546
[2.34 epo]: TRAIN:[loss=0.450] VALID:[RTE/accuracy=0.625]
Saving model a

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7fb3466d6410>
Traceback (most recent call last):
  File "/dfs/scratch0/vschen/venv-mmtl/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


RuntimeError: CUDA out of memory. Tried to allocate 46.88 MiB (GPU 0; 15.90 GiB total capacity; 8.62 GiB already allocated; 12.88 MiB free; 547.56 MiB cached)

In [ ]:
# Using GPU...
# [0.25060240963855424 epo]: TRAIN:[loss=0.475] VALID:[RTE/accuracy=0.671]
# Saving model at iteration 0.25060240963855424 with best (min) score 0.475
# Restoring best model from iteration 0.25060240963855424 with score 0.475
# Finished Training
# {'RTE/valid/accuracy': 0.6706827309236948}